In [1]:
from multiprocessing.sharedctypes import Value
import networkx as nx
import numpy as np
import random
import pandas as pd

from tqdm import tqdm
from datetime import datetime
import pickle

# File imports
import data
from data import select_dataset, get_station_g, ingest_electricity_data, set_random_speed_columns, ingest_pems
from replicate_graph import layer_graph
from vehicle import Vehicle
from simulation import Simulation
from visualization import set_draw_attributes

## Ingest pems for the whole graph (all charging and parking stops)
### commented out because it only needs to be done once. Csv already saved.

In [2]:
# all_graph = ingest_pems("data/charging_parking_stations_lonlat.csv", "data/charging_parking_distances.csv")

In [3]:
# #save the new csv with all the speeds (from pems ingest above), which will be used for the algorithm
# all_graph[1].to_csv("data/charging_parking_distances_pemsingested.csv")

## Run the algorithm

In [4]:
number_of_iterations = 5
kwh_per_km = 1.9

#scenario list: each list inside has three params : charging rate, kwh per km, and battery capacity.
# scenario_list = [[45,1.9,215],[90,1.9,215],[45,0.95,215],[45,1.9,430],[90,0.95,430]] 
scenario_list = [[90,0.95,430], [45,1.9,430], [45,0.95,215], [90,1.9,215], [45,1.9,215]] 

columns = ["scenario_number", "iteration", "charging_rate", "km_per_kwh", "battery_capacity", "success",
           "station_utilization_disp_of_avg", "station_utilization_avg_of_disp", "electricity",
           "percent_delay", "hours_spent_in_queues", "hours_spent_charging"]
results = [columns]

def print_and_record_results(sim, scenario, s, n, success=True):
    print("name: ", sim.name)
    print("success: ", success)
    
    if success==True:
        result = [s, n, scenario[0], scenario[1], scenario[2], True,
           sim.metrics['station_utilization_disp_of_avg'], sim.metrics['station_utilization_avg_of_disp'], 
           sim.metrics['electricity'], sim.metrics['percent_delay'],
           sim.metrics['hours_spent_in_queues'], sim.metrics['hours_spent_charging']]
        print("station_utilization_disp_of_avg:  ", sim.metrics['station_utilization_disp_of_avg'])
        print("station_utilization_avg_of_disp:  ", sim.metrics['station_utilization_avg_of_disp'])
        print("electricity:  ", sim.metrics['electricity'])
        print("percent_delay:  ", sim.metrics['percent_delay'])
        print("hours_spent_in_queues:  ", sim.metrics['hours_spent_in_queues'])
        print("hours_spent_charging:  ", sim.metrics['hours_spent_charging'])
        print("________________________________________")
        results.append(result)
    else:
        result = [s, n, scenario[0], scenario[1], scenario[2], False, None, None, None, None, None, None]
        results.append(result)

for scenario, s in zip(scenario_list, range(0,len(scenario_list))):
    
    print('------- SCENARIO '+str(s)+' -------')
    #initialize for wcctici
    simulation_length = 12
    battery_interval = 20
    kwh_per_km = scenario[1]
    battery_capacity = scenario[2]
    stations_path = "data/wcctci_stations-updated.csv"
    distances_path = "data/wcctci_coord_distances.csv"
    name = "scenario"+str(s)+"_wcctci" 
    
    for n in range(0, number_of_iterations):
        
        ### Print metrics
        sim = Simulation(name, stations_path, distances_path, simulation_length, battery_interval, kwh_per_km, battery_capacity)
        sim.add_demand_nodes()
        try:
            sim.run()
            print_and_record_results(sim, scenario, s, n, True)
        except:
            print_and_record_results(sim, scenario, s, n, False)
            continue

        ### get station utilization

        #get the station ids in order 
        current_stations = [int(j) for j in sim.station_g.nodes]

        #get the utilization rate of the stations
        utilization = sim.metrics['station_utilization']

        #get the station that has the highest utilization.
        h_st = current_stations[np.argmax(utilization)]

        #get the station that has the lowest utilization.
        l_st = current_stations[np.argmin(utilization)]


        ### Find the unused location that is closest to the highly utilized station

        #get the map of all parking areas and stations
        full_map_dist = pd.read_csv("data/charging_parking_distances_pemsingested.csv")
        full_map_loc = pd.read_csv("data/charging_parking_stations.csv")

        #keep only the roads that start or end with the high utilization station
        h_map = full_map_dist[(full_map_dist.loc[:, 'OriginID'] == h_st) | (full_map_dist.loc[:, 'DestinationID'] == h_st)]

        #then we eliminate all of the stations that are already being used.
        using_st = current_stations.copy()
        using_st.remove(h_st)
        h_map = h_map[(~ h_map.loc[:, 'OriginID'].isin(using_st)) & (~ h_map.loc[:, 'DestinationID'].isin(using_st))]

        #then we remove the case where we go from h_st to h_st
        h_map = h_map[~ ((h_map.loc[:, 'OriginID'] == h_st) & (h_map.loc[:, 'DestinationID'] == h_st))]

        #find the shortest edge. If there is more than one, we choose the first one that comes up (hence the 'reset_index' and [0] bellow)

        new_origin = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['OriginID'][0]
        new_destination = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['DestinationID'][0]

        #one of the two will be h_st and the other will be the new station location. 
        if new_origin == h_st:
            new_station = new_destination
        else:
            new_station = new_origin


        ### create the new maps with the locations and the distances of the stations that we will be using next

        new_stations_list = current_stations + [new_station]
        new_stations_list.remove(l_st)

        #now we reduce these to the new set of stations. All origins and destinations should be in this list.
        new_map_dist = full_map_dist[(full_map_dist.loc[:, 'OriginID'].isin(new_stations_list)) & (full_map_dist.loc[:, 'DestinationID'].isin(new_stations_list))]
        new_map_loc = full_map_loc[full_map_loc.loc[:, 'OID_'].isin(new_stations_list)]

        #we add the charging rates and capacity- assuming that they are the same as the others (set to 45 and 8)
        #TO DO: check this
        new_map_loc['charging_rate'] = scenario[0]
        new_map_loc['physical_capacity'] = 8

        #we change the snapX and snapY columns to lon and lat 
        new_map_loc = new_map_loc.rename(columns = {'SnapX': 'longitude', 'SnapY': 'latitude'})

        #now we save them. 
        new_map_loc.to_csv("data/algorithm_"+str(n+1)+"_stations.csv") 
        new_map_dist.to_csv("data/algorithm_"+str(n+1)+"_distances.csv")


        #reset the variables for the next iteration
        stations_path = "data/algorithm_"+str(n+1)+"_stations.csv"
        distances_path = "data/algorithm_"+str(n+1)+"_distances.csv"
        name = "anna_reverse_scenario_"+str(s)+"_algorithm_"+str(n+1)

    #we run the simulation for the final alocation

    ### run the simulation

    sim = Simulation(name, stations_path, distances_path, simulation_length, battery_interval, kwh_per_km, battery_capacity)
    sim.add_demand_nodes()
    
    try:
        sim.run()
        print_and_record_results(sim, scenario, s, n, True)
    except:
        print_and_record_results(sim, scenario, s, n, False)

pd.DataFrame(results).to_csv("algorithm_run_reverse_trial.csv")

------- SCENARIO 0 -------
km_per_percent get_station_g:  4.526315789473684
Strongly connected components: 3
Weakly connected components: 1
Weakly connected?: True


100%|██████████| 12/12 [00:34<00:00,  2.85s/it]


name:  scenario0_wcctci
success:  True
station_utilization_disp_of_avg:   4.2078125
station_utilization_avg_of_disp:   4.2421875
electricity:   0
percent_delay:   (1.0, 1.0, 1.4710974478242813, 1.0599016205399288, 0.0891474093331351)
hours_spent_in_queues:   (0.0, 0.0, 1.8, 0.014065934065934068, 0.12640400972547103)
hours_spent_charging:   (0, 0.0, 14, 0.4717948717948718, 2.0131734834350308)
________________________________________


/Users/alandler/.local/lib/python3.7/site-packages/ipykernel_launcher.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/alandler/.local/lib/python3.7/site-packages/ipykernel_launcher.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


km_per_percent get_station_g:  4.526315789473684
Strongly connected components: 2
Weakly connected components: 1
Weakly connected?: True


...

name:  anna_reverse_scenario_4_algorithm_3
success:  True
station_utilization_disp_of_avg:   15.930989583333332
station_utilization_avg_of_disp:   15.932552083333333
electricity:   0
percent_delay:   (1.2166331243308481, 1.5730789851367193, 5.352845055536708, 1.593142604108585, 0.6756084874205582)
hours_spent_in_queues:   (0.0, 0.0, 6.0, 0.2559055118110236, 1.0018214547974378)
hours_spent_charging:   (0, 0.0, 25, 1.1377952755905512, 4.107884278972144)
________________________________________
km_per_percent get_station_g:  1.131578947368421
Strongly connected components: 69
Weakly connected components: 1
Weakly connected?: True


100%|██████████| 12/12 [00:37<00:00,  3.11s/it]


name:  anna_reverse_scenario_4_algorithm_4
success:  True
station_utilization_disp_of_avg:   15.029166666666667
station_utilization_avg_of_disp:   15.04375
electricity:   0
percent_delay:   (1.2166331243308481, 1.4157710866230475, 5.082701833347463, 1.640312833288326, 0.7869534696573417)
hours_spent_in_queues:   (0.0, 0.0, 7.2, 0.3910569105691057, 1.26237021421483)
hours_spent_charging:   (0, 0.0, 26, 1.1666666666666667, 3.4817565575851592)
________________________________________
km_per_percent get_station_g:  1.131578947368421
Strongly connected components: 69
Weakly connected components: 2
Weakly connected?: False


  0%|          | 0/12 [00:02<?, ?it/s]

name:  anna_reverse_scenario_4_algorithm_5
success:  False


## Bellow this is  just code I used to think about/ build the algorithm above (Just ignore).
### I was thinking about just one iteration, starting from the wcctci pickle.

In [ ]:
#open the pickle
with open('trials/wcctci_04_04_2022_12_15_28.pkl', 'rb') as inp:
    res = pickle.load(inp)

Get the highest and lowest utilized stations:

In [ ]:
#get station utilization

#get the station ids in order 
current_stations = list(res.station_g.nodes)

#get the utilization rate of the stations
utilization = res.metrics['station_utilization']

#get the station that has the highest utilization.
h_st = current_stations[np.argmax(utilization)]

#get the station that has the lowest utilization.
l_st = current_stations[np.argmin(utilization)]

Find the unused location that is closest to the highly utilized station:

In [ ]:
#get the map of all parking areas and stations
full_map_dist = pd.read_csv("data/charging_parking_distances.csv")

#keep only the roads that start or end with the high utilization station
h_map = full_map_dist[(full_map_dist.loc[:, 'OriginID'] == h_st) | (full_map_dist.loc[:, 'DestinationID'] == h_st)]

#then we eliminate all of the stations that are already being used.
using_st = current_stations.copy()
using_st.remove(h_st)
h_map = h_map[(~ h_map.loc[:, 'OriginID'].isin(using_st)) & (~ h_map.loc[:, 'DestinationID'].isin(using_st))]

#then we remove the case where we go from h_st to h_st
h_map = h_map[~ ((h_map.loc[:, 'OriginID'] == h_st) & (h_map.loc[:, 'DestinationID'] == h_st))]

#find the shortest edge. If there is more than one, we choose the first one that comes up (hence the 'reset_index' and [0] bellow)

new_origin = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['OriginID'][0]
new_destination = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['DestinationID'][0]

#one of the two will be h_st and the other will be the new station location. 
if new_origin == h_st:
    new_station = new_destination
else:
    new_station = new_origin

Now we start to think about re-runing the simulation with this swap (taking out l_st and adding new_station). To do this, we need to create a new map with the locations and the distances of the stations that we will be using next.

In [ ]:
new_stations_list = current_stations + [new_station]

In [ ]:
#we have "full_map_dist" (distances map)
#and we need the locations map
full_map_loc = pd.read_csv("data/charging_parking_stations.csv")

#now we reduce these to the new set of stations. All origins and destinations should be in this list.
new_map_dist = full_map_dist[(full_map_dist.loc[:, 'OriginID'].isin(new_stations_list)) & (full_map_dist.loc[:, 'DestinationID'].isin(new_stations_list))]
new_map_loc = full_map_loc[(full_map_loc.loc[:, 'OriginID'].isin(new_stations_list)) & (full_map_loc.loc[:, 'DestinationID'].isin(new_stations_list))]

In [ ]:
#now we save them. 
new_map_loc.to_csv("data/algorithm_"+str(1)+"_stations.csv") ########change the 1
new_map_dist.to_csv("data/algorithm_"+str(1)+"_distances.csv")

Now we can run the next simulation

In [ ]:
simulation_length = 12
battery_interval = 20
km_per_percent = 3.13

sim = Simulation("data/algorithm_"+str(1)+"_stations.csv", "data/algorithm_"+str(1)+"_distances.csv", simulation_length, battery_interval, km_per_percent)

# Anna

In [ ]:
stations_path = "data/wcctci_stations-updated.csv"
distances_path = "data/wcctci_coord_distances.csv"
# stations_df, distances_df = select_dataset(stations_csv_path, distances_csv_path)
# station_g = get_station_g(stations_df, distances_df)
# battery_g = layer_graph(station_g, 20, km_per_percent= 1.9)
simulation_length = 24
battery_interval = 20
kwh_per_km = 1.9
battery_capacity = 215
name = 'hi'
sim = Simulation(name, stations_path, distances_path, simulation_length, battery_interval, kwh_per_km, battery_capacity)
sim.add_demand_nodes()

In [ ]:
sim.battery_g.nodes

In [ ]:
nx.shortest_path(sim.battery_g, "Long Beach", "Reno")